# Volcanic forcing

Use the AR6 process for volcanic forcing with updates:
- replace Toohey & Sigl (eVolv v2) with Sigl et al. (HolVol) which extends back to 9500 BC. We use the full "pre-industrial" (9500 BC to 1749 AD) as the average background sAOD to reference zero forcing to.
- ues GloSSAC v2.2.1 until the end of 2022
- Extend forwards to 2023 past the end of GlossAC using sAOD from OMPS LP, following the use of this dataset in https://www.nature.com/articles/s43247-022-00580-w#Sec11
- 2023 update to GloSSAC is coming and may replace the OMPS LP data if it arrives in time

This notebook requires large datasets that need registration to obtain so cannot be downloaded by the code:

- Download data from: https://download.pangaea.de/dataset/928646/files/HolVol_SAOD_-9500_1900_v1.0.nc Put this in '../data/volcanic_aod/HolVol_SAOD_-9500_1900_v1.0.nc'
- Download data from: https://asdc.larc.nasa.gov/project/GloSSAC/GloSSAC_2.21. Click "Get Dataset". Put this in '../data/volcanic_aod/GloSSAC_V2.21.nc'
- Download data from: https://omps.gesdisc.eosdis.nasa.gov/data/SNPP_OMPS_Level3/OMPS_NPP_LP_L3_AER_MONTHLY.1/. Obtain every *.h5 file in every annual sub-directory from 2013 to 2023. Put these files in '../data/volcanic_aod/SNPP_OMPS_Level3/

In addition, we require estimates of the stratospheric water vapour injection from Hunga Tonga. This does not appear in the actual calculation here. In 2022, we added +0.12 W/m2 from Jenkins et al. (2023). In the 2023 update, we take and process actual MLS data and run the RTM offline to get updated estimates of +0.14 W/m2 for 2022 and +0.18 W/m2 for 2023.

For bonus points we should consider stratospheric water vapour in historical eruptions, though this is not easy to get.

In [ ]:
import glob

from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
from pooch import retrieve
import scipy.stats
import h5py

In [ ]:
pl.rcParams['figure.figsize'] = (10/2.54, 10/2.54)
pl.rcParams['font.size'] = 11
pl.rcParams['font.family'] = 'Arial'
pl.rcParams['ytick.direction'] = 'in'
pl.rcParams['ytick.minor.visible'] = True
pl.rcParams['ytick.major.right'] = True
pl.rcParams['ytick.right'] = True
pl.rcParams['xtick.direction'] = 'in'
pl.rcParams['xtick.minor.visible'] = True
pl.rcParams['xtick.major.top'] = True
pl.rcParams['ytick.major.left'] = True
pl.rcParams['xtick.top'] = True
pl.rcParams['figure.dpi'] = 150
pl.rcParams['axes.spines.top'] = True
pl.rcParams['axes.spines.bottom'] = True

In [ ]:
# -9500 to 1900
nc = Dataset('../data/volcanic_aod/HolVol_SAOD_-9500_1900_v1.0.nc')
aod550_evolv = nc.variables['aod550'][:]
lat_evolv = nc.variables['lat'][:]
time_evolv = nc.variables['time'][:]
nc.close()
time_evolv[-51*12]

In [ ]:
lat_evolv_bnds = np.concatenate([[90], 0.5*(lat_evolv[1:]+lat_evolv[:-1]), [-90]])
weights = -np.squeeze(np.diff(np.sin(np.radians(lat_evolv_bnds))))
aod_evolv = np.zeros((len(time_evolv)))
for i in range(len(time_evolv)):
    aod_evolv[i] = np.average(aod550_evolv[i,:],weights=weights)

In [ ]:
# 1979 to 2022 from GloSSAC
nc = Dataset('../data/volcanic_aod/GloSSAC_V2.21.nc')
data_glossac = nc.variables['Glossac_Aerosol_Optical_Depth'][:]
lat_glossac = nc.variables['lat'][:]
trp_hgt_glossac = nc.variables['trp_hgt'][:]  # lat, month
alt_glossac = nc.variables['alt'][:]
nc.close()
data_glossac[0,0,:]

In [ ]:
lat_glossac_bnds = np.concatenate(([-90], 0.5*(lat_glossac[1:]+lat_glossac[:-1]), [90]))
weights_glossac = np.diff(np.sin(np.radians(lat_glossac_bnds)))

# Glossac is at 525 nm. -2.33 Angstrom exponent from Kovilakam et al 2020, https://essd.copernicus.org/articles/12/2607/2020/essd-12-2607-2020.html
angstrom = (550/525)**(-2.33)

months_glossac = 12*(2023-1979)
aod_glossac = np.zeros(months_glossac)
for i in range(months_glossac):
    aod_glossac[i] = np.average(data_glossac[i,:,2],weights=weights_glossac)*angstrom

In [ ]:
# 1850 to 2014 from CMIP6
# this file does not appear to be available from the FTP site any more. I'm using my AR6 copy
# cmip6_file = retrieve(
#     'ftp://iacftp.ethz.ch/pub_read/luo/CMIP6/CMIP_1850_2014_extinction_550nm_strat_only_v3.nc',
#     known_hash='f2cd708c9cd883e6ad0cc425d7bc6820a22639a752de9563bdc48a75c2550e4c'
# )
cmip6_file = '../data/volcanic_aod/CMIP_1850_2014_extinction_550nm_strat_only_v3.nc'

In [ ]:
nc = Dataset(cmip6_file)
ext_cmip6 = nc.variables['ext550'][:].transpose((2,1,0))  # time, height, lat
lev_cmip6 = nc.variables['altitude'][:]
lat_cmip6 = nc.variables['latitude'][:]
time_cmip6 = nc.variables['month'][:]
nc.close()

In [ ]:
time_cmip6

In [ ]:
lat_cmip6_bnds = np.concatenate(([-90], 0.5*(lat_cmip6[1:]+lat_cmip6[:-1]), [90]))
weights = np.diff(np.sin(np.radians(lat_cmip6_bnds)))
months_cmip6 = 12*(2015-1850)
tax = np.zeros(months_cmip6)
aod_cmip6 = np.zeros(months_cmip6)
for i in range(0,months_cmip6,12):
    gl_mn_aod = np.average(
        np.sum(
            np.mean(ext_cmip6[i:i+12,...], axis=0) * 0.5, axis=0),
        weights=weights
    ) # 0.5 is thickness in km

for i in range(months_cmip6):
    aod_cmip6[i] = np.average(np.sum(ext_cmip6[i,...] * 0.5, axis=0), weights=weights)

In [ ]:
aod_cmip6

In [ ]:
months_omps = 12 * (2024-2013)
aod_omps = np.zeros(months_omps)
aod_omps_unscaled = np.zeros(months_omps)

In [ ]:
# 745 nm band is index 3, and is the most reliable

# rather than try to estimate an Angstrom exponent, I will scale the timeseries to Glossac.


for i in range(months_omps):
    year = (i)//12 + 2013
    month = ((i-12)%12)+1
    filename = glob.glob('../data/volcanic_aod/SNPP_OMPS_Level3/OMPS-NPP_LP-L3-AER-MONTHLY_v1.0_%4dm%02d01_*.h5' % (year, month))[0]
    h5 = h5py.File(filename)
    lat_omps = h5['/']['Latitude'][:]
    lat_omps_bnds = np.concatenate(([-90], 0.5*(lat_omps[1:]+lat_omps[:-1]), [90]))
    weights = np.diff(np.sin(np.radians(lat_omps_bnds)))
    data = h5['/']['StratColumn'][:]
    data[data==-999] = np.nan
    aod_omps_unscaled[i] = np.nansum(data[:,:,3] * weights * np.ones((24, 36)))/((weights * np.ones((24, 36)))[~np.isnan(data[:,:,3])].sum())

In [ ]:
#aod_omps = aod_glossac[-108:].mean()/aod_omps_unscaled[:108].mean() * aod_omps_unscaled
# calculate scaling factor between OMPS and GloSSAC for the overlap period 2013-2022
aod_omps = aod_glossac[-(months_omps-12):].mean()/aod_omps_unscaled[:(months_omps-12)].mean() * aod_omps_unscaled

In [ ]:
# eVolv -9500 to 1849
# CMIP6 1850 to 1978
# Glossac 1979 to 2022
# OMPS 2023
aod = np.concatenate((aod_evolv[:-51*12], aod_cmip6[:129*12], aod_glossac, aod_omps[-12:]))
len(aod)

In [ ]:
pl.plot(np.arange(2013+1/24, 2023, 1/12), aod_glossac[-(months_omps-12):], label='GloSSAC v2.2.1 (550 nm)')
pl.plot(np.arange(2013+1/24, 2024, 1/12), aod_omps_unscaled, label='OMPS LP (745 nm)')
pl.plot(np.arange(2013+1/24, 2024, 1/12), aod_omps, label='OMPS LP scaled')
pl.ylabel('sAOD')
pl.legend()
#pl.plot(aod[-120:])

In [ ]:
# crossfade
aod[136200:136812] = (1-np.linspace(0,1,612))*aod_evolv[-51*12:]+np.linspace(0,1,612)*aod_cmip6[:612]
aod[137748:137868] = (1-np.linspace(0,1,120))*aod_cmip6[1548:1668] + np.linspace(0,1,120)*aod_glossac[:120]
aod[138180:138276] = (1-np.linspace(0,1,96))*aod_glossac[-96:] + np.linspace(0, 1, 96)*aod_omps[24:120]

In [ ]:
pl.plot(np.arange(1845+1/24,1901+1/24,1/12), aod_evolv[136140:], label='HolVol')
pl.plot(np.arange(1850+1/24,1905+1/24,1/12), aod_cmip6[:660], label='CMIP6')
pl.plot(np.arange(1845+1/24,1905+1/24,1/12), aod[136140:136860], label='blended')
pl.legend()

In [ ]:
pl.plot(np.arange(1979+1/24,2023+1/24,1/12), aod_glossac, label='GloSSAC v2.2.1 (1979-2022)', lw=0.5)
pl.plot(np.arange(1975+1/24,2015+1/24,1/12), aod_cmip6[-480:], label='CMIP6 (1850-2014)', lw=0.5)
pl.plot(np.arange(2013+1/24,2024+1/24,1/12), aod_omps, label='OMPS (2013-2023)', lw=0.5)
pl.plot(np.arange(1975+1/24,2024+1/24,1/12), aod[-588:], label='Combined', zorder=-2, color='k')
pl.title('Stratospheric aerosol optical depth')
pl.ylabel("AOD")
pl.xlim(1975, 2024)
pl.ylim(0, 0.12)
pl.tight_layout()
pl.legend(frameon=False, fontsize=8)
pl.tight_layout()
pl.savefig('../plots/volcanic_AOD.png')
pl.savefig('../plots/volcanic_AOD.pdf')

In [ ]:
erf_sulf = -20 * (aod - np.mean(aod[:(11250*12)]))
erf_h2o = np.zeros_like(erf_sulf)

# hunga tonga adjustment - this comes from an offline radiative transfer simulation (but everything prepared here)
erf_h2o[-24:] = np.array([
    0.056142705,
    0.124763041,
    0.134787523,
    0.140017655,
    0.148023525,
    0.149671158,
    0.148921003,
    0.145035717,
    0.146799994,
    0.154511014,
    0.165538079,
    0.184053622,
    0.19959181,
    0.201186799,
    0.189308977,
    0.187860589,
    0.188655669,
    0.17539622,
    0.164919464,
    0.168585382,
    0.166695728,
    0.175021385,
    0.181187582,
    0.185611578,
])

erf = erf_sulf + erf_h2o

In [ ]:
pl.plot(np.arange(1975+1/24,2024+1/24,1/12), erf_sulf[-(12*49):], label='AOD', color='skyblue')
pl.plot(np.arange(2021+23/24,2024,1/12), erf[-25:], label='AOD + H$_2$O', color='purple')
pl.title('Volcanic effective radiative forcing')
pl.xlim(1975, 2024)
pl.ylim(-2.1, 0.3)
pl.ylabel('W m$^{-2}$, relative to 9500 BCE to 1749 CE')
pl.axhline(0, ls=':', color='k')
pl.legend(frameon=False)
pl.tight_layout()
pl.savefig('../plots/volcanic_ERF.png')
pl.savefig('../plots/volcanic_ERF.pdf')

In [ ]:
months = np.arange(-9500+1/24,2024,1/12)
df = pd.DataFrame(
    data=np.vstack([aod, erf_sulf, erf_h2o, erf]).T, 
    index=months, 
    columns=['stratospheric_AOD', 'ERF_sulfate', 'ERF_H2O', 'volcanic_ERF']
)
df.index.name = 'year'
df.to_csv('../output/volcanic_sAOD_ERF_monthly_-950001-202312.csv')

In [ ]:
years = np.arange(-9500 + 0.5, 2024)
aod_ann = np.zeros(len(aod)//12)
sul_ann = np.zeros(len(erf)//12)
h2o_ann = np.zeros(len(erf)//12)
erf_ann = np.zeros(len(erf)//12)
for i in range(0, len(months), 12):
    aod_ann[i//12] = np.mean(aod[i:i+12])
    sul_ann[i//12] = np.mean(erf_sulf[i:i+12])
    h2o_ann[i//12] = np.mean(erf_h2o[i:i+12])
    erf_ann[i//12] = np.mean(erf[i:i+12])
df = pd.DataFrame(data=np.vstack([aod_ann, sul_ann, h2o_ann, erf_ann]).T, index=years, columns=['stratospheric_AOD', 'ERF_sulfate', 'ERF_H2O', 'volcanic_ERF'])
df.index.name = 'year'
df.to_csv('../output/volcanic_sAOD_ERF_annual_-9500-2023.csv')

In [ ]:
df

In [ ]:
pl.plot(df.volcanic_ERF)